Topic classification with BERTopic

Install transformers 4.41.0 for compatability with spacy and BERTopic

`pip install transformers==4.41.0` 

Careful when running this, very hardware intensive

In [1]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import psutil
import gc
from tqdm import tqdm
import re

/Users/lauritseisengarten/Documents/GitHub/case-study-ul/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_posts = pd.read_json('../data/dataset.json')
df_posts.head()

,timestamp,text,text_id,user,user_id
0,2024-10-31,Running a business means juggling countless ad...,2018569761,danielwoodard,1077866112
1,2024-10-31,Liz Truss is walking in the lingering shadow o...,2092717718,nelsonjacqueline,1089670430
2,2024-10-31,The UK is bracing for war as government buildi...,2059143248,ihooper,1007478642
3,2024-10-31,Marrying a second or third cousin once removed...,2008209828,wrightnicholas,1039258480
4,2024-10-31,It's truly disgraceful how the Indian National...,2001239278,michael51,1021455936


Apply minimal preprocessing (Remove URLs and Mentions)

In [3]:
df_posts['text'] = df_posts['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+|@\w+', '', x) if pd.notna(x) else x)


In [4]:
#df_posts = df_posts.sample(n=10000, random_state=42)  # Uncomment to test with sample

In [5]:
def monitor_memory():
    """Monitor memory usage"""
    process = psutil.Process()
    memory_gb = process.memory_info().rss / 1024 / 1024 / 1024
    return f"Memory Usage: {memory_gb:.2f} GB"

def create_multifeature_embeddings(df_posts, sentence_model, batch_size=64):
    """
    Create embeddings with memory monitoring and larger batches
    """
    print(f"\nStarting embedding generation for {len(df_posts)} documents")
    print(monitor_memory())
    
    print("\nGenerating text embeddings...")
    text_embeddings = sentence_model.encode(
        df_posts['text'].fillna("").tolist(),
        batch_size=batch_size,
        show_progress_bar=True
    )
    print(monitor_memory())
    
    print(monitor_memory())
    return text_embeddings

def setup_bertopic_model(df_posts, batch_size=64):
    """
    Set up and train BERTopic model with memory optimization
    """
    print(f"\nDataset size: {len(df_posts)} documents")
    print(f"DataFrame memory usage: {df_posts.memory_usage().sum() / 1024**2:.2f} MB")
    print(monitor_memory())
    
    print("\nInitializing models...")
    sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
    
    vectorizer_model = CountVectorizer(
        stop_words="english",
        #min_df=2, 
        #max_df=0.8
        ngram_range=(1, 2)
    )
    
    umap_model = UMAP(
        n_neighbors=15,
        n_components=5,
        min_dist=0.0,
        metric='cosine',
        low_memory=True,
        random_state=42
    )
    
    embeddings = create_multifeature_embeddings(df_posts, sentence_model, batch_size)
    
    topic_model = BERTopic(
        embedding_model=sentence_model,
        vectorizer_model=vectorizer_model,
        umap_model=umap_model,
        min_topic_size=30,
        nr_topics='auto',
        calculate_probabilities=True,
        verbose=True
    )
    
    print("\nFitting BERTopic model...")
    topics, probs = topic_model.fit_transform(
        documents=df_posts['text'].fillna("").tolist(),
        embeddings=embeddings
    )
    
    return topic_model, topics, probs

def analyze_topics(topic_model, topics, df_posts):
    """
    Analyze topics with memory considerations
    """
    print("\nAnalyzing topics...")
    print(monitor_memory())
    
    topic_info = topic_model.get_topic_info()
    
    df_posts['topic'] = topics
    
    topic_docs = {}
    unique_topics = set(topics)
    print(f"\nFound {len(unique_topics)-1} topics (excluding -1)")
    
    for topic in tqdm(unique_topics):
        if topic != -1:
            topic_docs[topic] = df_posts[df_posts['topic'] == topic]['text'].head(3).tolist()
    
    try:
        print("\nGenerating visualizations...")
        topic_model.visualize_topics()
        topic_model.visualize_hierarchy()
    except Exception as e:
        print(f"Warning: Visualization error: {e}")
    
    return topic_info, topic_docs

def run_topic_analysis(df_posts, batch_size=64):
    """
    Run the complete pipeline with memory monitoring
    """
    print(f"Starting analysis with batch size: {batch_size}")
    print(monitor_memory())
    
    if 'text' not in df_posts.columns:
        raise ValueError("Missing required column: text")
    
    topic_model, topics, probs = setup_bertopic_model(df_posts, batch_size)
    topic_info, topic_docs = analyze_topics(topic_model, topics, df_posts)
    
    summary = {
        'num_topics': len(set(topics)) - 1,
        'topic_sizes': topic_info['Count'].tolist(),
        'top_topics': topic_info.head(10).to_dict('records')
    }
    
    return topic_model, summary, topics, probs

In [6]:
try:
    topic_model, summary, topics, probs = run_topic_analysis(df_posts, batch_size=64)
    
    print(f"\nAnalysis complete!")
    print(f"Found {summary['num_topics']} topics")
    print("\nTop 10 topics:")
    for topic in summary['top_topics']:
        print(f"Topic {topic['Topic']}: Size {topic['Count']}")
    
    topic_model.save("bertopic_model_large")
    
except Exception as e:
    print(f"Error during analysis: {e}")

Starting analysis with batch size: 64
Memory Usage: 0.67 GB

Dataset size: 70260 documents
DataFrame memory usage: 2.68 MB
Memory Usage: 0.67 GB

Initializing models...

Starting embedding generation for 70260 documents
Memory Usage: 0.77 GB

Generating text embeddings...


Batches: 100%|██████████| 1098/1098 [01:40<00:00, 10.96it/s]
2025-01-23 14:22:09,282 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Memory Usage: 1.46 GB
Memory Usage: 1.46 GB

Fitting BERTopic model...


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2025-01-23 14:22:55,798 - BERTopic - Dimensionality - Completed ✓
2025-01-23 14:22:55,799 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. 


Analyzing topics...
Memory Usage: 1.35 GB

Found 273 topics (excluding -1)


100%|██████████| 274/274 [00:00<00:00, 6656.82it/s]


Generating visualizations...



2025-01-23 14:26:40,100 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.



Analysis complete!
Found 273 topics

Top 10 topics:
Topic -1: Size 28435
Topic 0: Size 3280
Topic 1: Size 2172
Topic 2: Size 1992
Topic 3: Size 1967
Topic 4: Size 1468
Topic 5: Size 1037
Topic 6: Size 924
Topic 7: Size 807
Topic 8: Size 778


In [7]:
# 1. Save visualizations to HTML files
fig = topic_model.visualize_barchart(top_n_topics=20)
fig.write_html("../output/topic_barchart.html")

topic_model.visualize_topics().write_html("../output/topic_clusters.html")
topic_model.visualize_hierarchy().write_html("../output/topic_hierarchy.html")

# 2. Print text-based summary
topics_info = topic_model.get_topic_info()
print("\nMost frequent topics with their terms:")
for _, row in topics_info.head(20).iterrows():
    topic_id = row['Topic']
    size = row['Count']
    if topic_id != -1:
        terms = topic_model.get_topic(topic_id)
        print(f"\nTopic {topic_id} (Size: {size}):")
        # Print top 10 terms for each topic with their weights
        for term, weight in terms[:20]:
            print(f"  - {term}: {weight:.3f}")


Most frequent topics with their terms:

Topic 0 (Size: 3280):
  - vote: 0.024
  - btsbbmas: 0.023
  - social artist: 0.022
  - artist: 0.020
  - voting: 0.019
  - bbmas: 0.019
  - social: 0.018
  - army: 0.018
  - counts: 0.016
  - vote counts: 0.015

Topic 1 (Size: 2172):
  - secretary: 0.016
  - health secretary: 0.016
  - health: 0.013
  - healthcare: 0.012
  - coffey: 0.012
  - nhs: 0.011
  - healthsecretary: 0.009
  - new health: 0.008
  - minister: 0.007
  - medical: 0.007

Topic 2 (Size: 1992):
  - mental: 0.035
  - mental health: 0.034
  - mentalhealth: 0.021
  - mentalhealthawareness: 0.020
  - health: 0.013
  - mentalhealthmatters: 0.013
  - selfcare: 0.011
  - important: 0.009
  - care: 0.008
  - prioritize: 0.007

Topic 3 (Size: 1967):
  - politics: 0.015
  - unity: 0.007
  - political: 0.007
  - truth: 0.006
  - journalism: 0.006
  - truthmatters: 0.006
  - accountability: 0.006
  - people: 0.006
  - time: 0.006
  - tories: 0.005

Topic 4 (Size: 1468):
  - business: 0.014

In [8]:
# Add topic numbers and labels to dataframe 
df_posts['topic'] = topics

# Get topic labels
topic_labels = {}
for topic_id in set(topics):
    if topic_id != -1:
        terms = topic_model.get_topic(topic_id)
        topic_labels[topic_id] = ", ".join([term for term, _ in terms[:3]])
    else:
        topic_labels[topic_id] = "Other"

# Add descriptive labels
df_posts['topic_label'] = df_posts['topic'].map(topic_labels)

# Save augmented dataframe
df_posts.to_csv('../output/posts_with_topics.csv', index=False)